In [1]:
"""
Configs:
    # Env
    env_id="NNWorld01-v01", 
     - once in the goal state, each action ends the episode and returns a reward of 5
     - no clear episodes, continuously train the agent with num_train_steps, 
       if the agent reaches terminal, just reset environment and keep training 
    # Params
    eps_sched_fn=poly(0.5), lr_sched_fn=poly(0.8)
    # Algos
    haver2, action_sigma=adaptive(1), haver_delta=0.01, haver_const=varied
Status:
"""

from collections import defaultdict
import random
import numpy as np
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")
sns.set_palette("tab20")
colors = sns.color_palette("bright")

import time
from tqdm import tqdm 
import multiprocessing

# import gymnasium as gym
import gym
import gym_examples
from gym.wrappers import FlattenObservation

from algos import *
from bandit_problem import *
from utils import *

In [ ]:

random.seed(123)
np.random.seed(123)
tdqm_disable = True

# params
num_trials = 200
num_steps_train = 5000
num_episodes_eval = 100

lr_sched_type = "poly"
lr_sched_fn = create_lr_sched_fn(lr_sched_type, lr=0.7)

max_eps = 1.0
min_eps = 0.01
decay_rate = 0.0001
eps_sched_type = "poly"
eps_sched_fn = create_eps_sched_fn(eps_sched_type, min_eps, max_eps, decay_rate)

haver_const = 1.0
est_name = "haver" 
q_algo_name = "haver_q_learning"
q_algo = create_q_algo(q_algo_name)

# create gym env
env_id = "gym_examples/NNWorldEnv01-v1"
gamma = 0.95

num_depths = 4
num_widths = 32
num_actions = num_widths
terminal_reward = 20.0

reward_dist = "normal"
problem_instance = "multi_gap_nonlinear"
action_max_mu = -1.0
action_sigma = 1.0
action_sigmas = action_sigma*np.ones(num_actions)
gap_splits = [0.5]
gap_deltas = [1.0]
bandit_problem = BanditProblem(
    problem_instance, reward_dist, num_actions, action_max_mu, 
    action_sigmas=action_sigmas, gap_splits=gap_splits, gap_deltas=gap_deltas)
print(f"action_mus = {bandit_problem.action_mus}")
print(f"action_sigmas = {bandit_problem.action_sigmas}")

env = gym.make(env_id, num_depths=num_depths, num_widths=num_widths, 
               bandit_problem=bandit_problem, terminal_reward=terminal_reward)
env_wrapped = FlattenObservation(env)
cur_state, info = env_wrapped.reset()

manager = multiprocessing.Manager()
episode_start_sigmahats_list = manager.list()
episode_rewards_list = manager.list()
episode_start_muhats_list = manager.list()
Q_table_list = manager.list()
Q_nvisits_list = manager.list()    

def run_trial(i_trial, args):

    random.seed(10000+i_trial)
    np.random.seed(10000+i_trial)

    # env = gym.make(env_id, size=gridworld_size)
    # env_wrapped = FlattenObservation(env)
    # env_wrapped.reset(seed=10000+i_trial)

    # lr_sched_fn = create_lr_sched_fn(lr_sched_type)
    # eps_sched_fn = create_eps_sched_fn(eps_sched_type, min_eps, max_eps, decay_rate)
    q_algo = create_q_algo(args["est_name"])

    Q_table, Q_nvisits, stats = q_algo(
        env_wrapped, num_actions, num_steps_train,
        gamma, lr_sched_fn, eps_sched_fn, tdqm_disable, args)

    episode_start_sigmahats, episode_rewards, episode_start_muhats= zip(*stats)
    episode_start_sigmahats_list.append(episode_start_sigmahats)
    episode_rewards_list.append(episode_rewards)
    episode_start_muhats_list.append(episode_start_muhats)
    Q_table_list.append(Q_table)
    Q_nvisits_list.append(Q_nvisits)

args = dict()
args["action_sigma"] = 1
args["haver_alpha"] = 2.0
args["haver_delta"] = 0.05
args["haver_const"] = 1.0
args["weightedms_num_data"] = 1000

pool = multiprocessing.Pool()

episode_start_sigmahats_mean_dict = defaultdict()
episode_rewards_mean_dict = defaultdict()
episode_start_muhats_mean_dict = defaultdict()
Q_table_dict = defaultdict()
Q_nvisits_dict = defaultdict()

haver_const_ary = [1.0]
haver_name_ary = [f"haver_{x}" for x in haver_const_ary]

est_name_ary = ["weightedms", "double", "max"]
est_name_ary = est_name_ary + haver_name_ary
# est_name_ary = ["max"]
for est_name in est_name_ary:
    start_time = time.time()
    print(f"\n-> est_name = {est_name}")
    if "haver" in est_name:
        args["est_name"] = "haver2"
        args["haver_const"] = float(est_name.split("_")[-1])
        print(f"haver_const = {args['haver_const']}")
    else:
        args["est_name"] = est_name
    
    pool.starmap(run_trial, [(i, args) for i in range(num_trials)])

    episode_start_sigmahats_ary = np.hstack([episode_start_sigmahats_list])
    episode_rewards_ary = np.hstack([episode_rewards_list])
    episode_start_muhats_ary = np.hstack([episode_start_muhats_list])

    episode_start_sigmahats_mean_dict[est_name] = np.mean(episode_start_sigmahats_ary, 0)
    episode_rewards_mean_dict[est_name] = np.mean(episode_rewards_ary, 0)
    episode_start_muhats_mean_dict[est_name] = np.mean(episode_start_muhats_ary, 0)
    print(f"last_episode_start_sigmahat = {episode_start_sigmahats_mean_dict[est_name][-1]:.4f}")
    print(f"last_episode_reward_per_step = {episode_rewards_mean_dict[est_name][-1]:.4f}")
    print(f"last_episode_estim_start_muhat = {episode_start_muhats_mean_dict[est_name][-1]:.4f}")
    
    Q_table_dict[est_name] = np.mean(np.stack(Q_table_list), 0)
    Q_nvisits_dict[est_name] = np.mean(np.stack(Q_nvisits_list), 0)
    # print(Q_table_list[0][0,0,:])
    # print(Q_table_list[1][0,0,:])
    # print(Q_table_ary)
    # stop
                           
    episode_start_sigmahats_list[:] = []
    episode_rewards_list[:] = []
    episode_start_muhats_list[:] = []
    Q_table_list[:] = []
    Q_nvisits_list[:] = []
    
    end_time = time.time()
    print(f"it takes {end_time-start_time:0.4f}")

action_mus = [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -2. -2.
 -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
action_sigmas = [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]

-> est_name = weightedms
last_episode_start_sigmahat = 1250.0000
last_episode_reward_per_step = 20.0000
last_episode_estim_start_muhat = -0.0018
it takes 83.9872

-> est_name = double


In [ ]:

fig, axes = fig, axes = plt.subplots(
        nrows=2, ncols=1, sharex=True, sharey=False, figsize=(8,8))
# axes = [axes]

x_ary = np.linspace(0, num_steps_train-1, num=100, dtype=np.int32)
for est_name in est_name_ary[:4]:
    # axes[0].plot(x_ary, episode_rewards_mean_dict[est_name][x_ary], label=est_name)
    y_ary = running_avg(episode_rewards_mean_dict[est_name][x_ary], 100)
    axes[0].plot(
        x_ary, y_ary, label=est_name)
    axes[1].plot(x_ary, episode_start_muhats_mean_dict[est_name][x_ary], label=est_name)
    # axes[2].plot(x_ary, episode_start_sigmahats_mean_dict[est_name][x_ary], label=est_name)

action_max_mu = bandit_problem.action_mus[0]
optimal_num_steps = num_depths
optimal_vstar = terminal_reward*gamma**(optimal_num_steps-1) \
    + action_max_mu*np.sum([gamma**k for k in range(optimal_num_steps-1)])
optimal_reward_per_step = (terminal_reward + action_max_mu*(optimal_num_steps-1))/optimal_num_steps  
print(optimal_num_steps)
print(optimal_reward_per_step)
print(optimal_vstar)
axes[0].axhline(y=optimal_reward_per_step, color="black")
axes[1].axhline(y=optimal_vstar, color="black")
axes[0].set_title(f"reward_per_step")
axes[1].set_title("estim_qval_start_state")
# axes[2].set_title("episode_length")
axes[0].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
axes[1].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# axes[2].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.show()

In [ ]:
# fig, axes = fig, axes = plt.subplots(
#         nrows=1, ncols=1, sharex=True, sharey=False, figsize=(8,8))
# axes = [axes]

# x_ary = np.linspace(2000, num_steps_train-1, num=1000, dtype=np.int32)
# # est_name_ary2 = haver_name_ary + ["weightedms"]
# for est_name in est_name_ary:
#     axes[0].plot(x_ary, episode_start_sigmahats_mean_dict[est_name][x_ary], label=est_name)

# axes[0].set_title("Q_sigmahat")
# # axes[0].set_ylim([0, 1])
# axes[0].legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# plt.show()

In [ ]:

for est_name in est_name_ary:
    print(f"\n-> est_name = {est_name}")
    Q_table = Q_table_dict[est_name]
    Q_nvisits = Q_nvisits_dict[est_name]
    for i_row in range(num_depths):
        for j_col in range(num_actions):
            print(f"{i_row} {j_col}: {Q_table[i_row,j_col]}, {Q_nvisits[i_row,j_col]}")
